In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import umap
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from deeptime.models.oneclass.linear import LinearOCC
from deeptime.models.representation import LinearAutoEncoder
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset

from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
DATASET = 'ECGFiveDays'
ACTIVATION = 'swish'
LABEL = 2
LATENT = 32

In [3]:
x_train, y_train = load_UCR_UEA_dataset(name=DATASET, split='train')
# Since the features from the sktime are instatiated as objects we have to manually convert them
y_train = np.array(y_train, dtype=np.int32)

sequence_length = x_train.values[0][0].shape[0]

x_test, y_test = load_UCR_UEA_dataset(name=DATASET, split='test')
y_test = np.array(y_test, dtype='int32') # Fixing the labels type

x_train_transformed = []
for val in x_train.values:
    x_train_transformed.append(val[0].tolist())
x_train = np.array(x_train_transformed)

x_test_transformed = []
for val in x_test.values:
    x_test_transformed.append(val[0].tolist())
x_test = np.array(x_test_transformed)

x_train = x_train[y_train == LABEL]
y_train = y_train[y_train == LABEL]

train_dataset = BaseDataset(x=x_train, y=y_train)
train_loader = DataLoader(train_dataset, batch_size=32)

model = LinearAutoEncoder(input_dim=sequence_length, latent_dim=LATENT, activation='relu')
model.load_state_dict(torch.load(f'../../../pretrain/representation/{DATASET}/linear_autoencoder-dim={LATENT}-l={LABEL}.pt'))

<All keys matched successfully>

In [4]:
y_test = np.array([1 if lbl == LABEL else -1 for lbl in y_test])
test_dataset = BaseDataset(x=x_test, y=y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
representations = []

for x, _ in train_loader:
    _, z = model(x)
    representations.extend(z.tolist())
representations = np.array(representations)

center = representations.mean(axis=0)
center

array([1.89579384, 0.        , 1.89928275, 1.57951126, 1.71423286,
       0.11256527, 0.        , 2.47822709, 0.0947147 , 1.5675701 ,
       1.04553103, 0.        , 1.03788583, 0.02054047, 0.70356057,
       3.09777347, 2.19886301, 3.36768095, 0.        , 1.97289443,
       0.99083355, 0.02961204, 1.06661824, 1.73400725, 1.73524986,
       2.44972973, 2.8294953 , 3.14087472, 0.94983392, 0.95973   ,
       2.35997955, 0.        ])

In [6]:
class Plot2DCallback(pl.Callback):

    def __init__(self, path: str, label) -> None:
        super().__init__()
        self.path = path
        self.label = label

    def on_train_epoch_end(
        self,
        trainer: pl.Trainer,
        pl_module: pl.LightningModule
    ) -> None:
        fig, ax = plt.subplots(figsize=(10, 10))

        circle = plt.Circle((pl_module.center[0].item(), pl_module.center[1].item()), 2, fill=False, color='red')
        ax.add_patch(circle)

        colors = []
        
        ax.set_ylim([-10, 10])
        ax.set_xlim([-10, 10])

        representations = []
        for x, y in trainer.train_dataloader:
            x = x.view(x.shape[0], -1)
            z = pl_module(x.to('cuda'))

            representations.extend(z.tolist())
            colors.extend(['lightgreen' for _ in y])

        test_representations = []
        for x, y in trainer.val_dataloaders[0]:
            x = x.view(x.shape[0], -1)
            z = pl_module(x.to('cuda'))

            test_representations.extend(z.tolist())

            colors.extend(['lightgreen' if l == self.label else 'tomato' for l in y])

        representations.extend(test_representations)
        representations = np.array(representations)

        ax.scatter(representations[:, 0], representations[:, 1], c=colors)

        plt.savefig(os.path.join(self.path, f'{trainer.current_epoch}.png'))
        plt.close()
        
class LitProgressBar(pl.callbacks.TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(
            disable=True,
        )
        return bar

In [7]:
model = LinearOCC(
    input_dim=sequence_length,
    latent_dim=LATENT,
    activation=ACTIVATION,
    radius=3,
    learning_rate=5e-5
)

model.center = torch.from_numpy(center)
# model.center = torch.tensor([0., 0.]).float()

# model.load_pretrained_weights(
#     f'../../../pretrain/representation/{DATASET}/linear_autoencoder-dim={LATENT}-l={LABEL}.pt'
# )

In [8]:
trainer = pl.Trainer(
    max_epochs=500,
    accelerator='gpu',
    devices=-1,
    callbacks=[
        # Plot2DCallback(path='./logs/linear/', label=LABEL),
        LitProgressBar()
    ]
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | e    | Sequential | 334 K 
------------------------------------
334 K     Trainable params
0         Non-trainable params
334 K     Total params
1.336     Total estimated model params size (MB)


Epoch 112:  82%|████████▏ | 23/28 [00:00<00:00, 73.24it/s, loss=0.467, v_num=166, train_loss=0.000, val_f1_score=0.676]

In [9]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 27it [00:00, 41.35it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5435540069686411     │
│       test_f1_score       │    0.6768442958752664     │
│   test_precision_score    │     0.526581414488276     │
│     test_recall_score     │    0.9644118038029647     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.5435540069686411,
  'test_f1_score': 0.6768442958752664,
  'test_recall_score': 0.9644118038029647,
  'test_precision_score': 0.526581414488276}]